# Data Discovery with NASA's CMR Using `requests` Python Library

## Summary

In this notebook, we will walk through how to search for Earthdata data collections and granules. Along the way we will explore the available search parameters, information return, and specific contrains when using the CMR API. Our object is to identify assets to access that we would downloaded, or perform S3 direct access, within an analysis workflow 


## Learning Objectives

- Understand what CMR/CMR API is and what CMR/CMR API can be used for 
- How to use the `requests` library to search data collections and granules
- How to parse the results of these searches.

## What is CMR
CMR is the Common Metadata Repository.  It catalogs all data for NASA's Earth Observing System Data and Information System (EOSDIS).  It is the backend of [Earthdata Search](https://search.earthdata.nasa.gov/search), the GUI search interface.  More information about CMR can be found [here](https://earthdata.nasa.gov/eosdis/science-system-description/eosdis-components/cmr).

Unfortunately, the GUI for Earthdata Search is not accessible from a cloud instance - at least not without some work.  Earthdata Search is also not immediately reproducible.  What I mean by that is if you create a search using the GUI you would have to note the search criteria (date range, search area, collection name, etc), take a screenshot, copy the search url, or save the list of data granules returned by the search, in order to recreate the search.  This information would have to be re-entered each time you or someone else wanted to do the search.  You could make typos or other mistakes.  A cleaner, reproducible solution is to search CMR programmatically using the CMR API.

## What is the CMR API
API stands for Application Programming Interface.  It allows applications (software, services, etc) to send information to each other.  A helpful analogy is a waiter in a restaurant.  The waiter takes your drink or food order that you select from the menu, often translated into short-hand, to the bar or kitchen, and then returns (hopefully) with what you ordered when it is ready.

The CMR API accepts search terms such as collection name, keywords, datetime range, and location, queries the CMR database and returns the results.

---

## Getting Started: How to search CMR from Python
The first step is to import python libraries.  We will use:  
- `requests` This library does most of the work for us accessing the CMR API using HTTP methods.
- `pprint` to _pretty print_ the results of the search.  

A more in-depth tutorial on `requests` is [here](https://realpython.com/python-requests/)

In [1]:
import requests
from pprint import pprint

To conduct a search using the CMR API, `requests` needs the url for the root CMR search endpoint. We'll assign this url to a python variable as a _string_.

In [2]:
CMR_OPS = 'https://cmr.earthdata.nasa.gov/search'

# Searching for Collections

CMR allows search by __collections__, which are datasets, and __granules__, which are files that contain data. Many of the same search parameters can be used for collections and granules but the type of results returned differ. Search parameters can be found in the [API Documentation](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html).  

Whether we search __collections__ or __granules__ is distinguished by adding `"collections"` or `"granules"` to the end of the CMR endpoint URL.  

We are going to search collections first, so we add `"collections"` to the URL. We are using a `python` format string in the examples below.

In [3]:
url = f'{CMR_OPS}/{"collections"}'
url

'https://cmr.earthdata.nasa.gov/search/collections'

## Query CMR for Cloud Hosted Collections

In this tutorial, we use different search parameters to search for collections in different ways. Below, we want to retrieve the collections that are hosted in the cloud (`'cloud_hosted': 'True'`) that has granules availble (`'has_granules': 'True'`). We also want to get the content in `json` (pronounced "jason") format, so I pass a dictionary to the header keyword argument to say that I want results returned as `json` (`'Accept': 'application/json'`).

The `.get()` method is used to send this information to the CMR API. `get()` calls the HTTP method __GET__. 

In [4]:
response = requests.get(url,
                        params={
                            'cloud_hosted': 'True',
                            'has_granules': 'True',
                        },
                        headers={
                            'Accept': 'application/json'
                        }
                       )

The request returns a `Response` object.    

To check that our request was successful we can print the `response` variable we saved the request to.

In [5]:
response

<Response [200]>

A __200__ response is what we want. This means that the requests was successful. For more information on HTTP status codes see <https://en.wikipedia.org/wiki/List_of_HTTP_status_codes>

A more explict way to check the status code is to use the `status_code` attribute. Both methods return a HTTP status code.

In [6]:
response.status_code

200

The response from `requests.get` returns the results of the search and metadata about those results in the `headers`.  

More information about the `response` object can be found by typing `help(response)`.

`headers` contains useful information in a case-insensitive dictionary. We requested (above) that the information be return in json which means the object return is a dictionary in our Python environment. We'll iterate through the returned dictionary, looping throught each field (`k`) and its associated value (`v`). For more on interating through dictionary object click [here](https://realpython.com/iterate-through-dictionary-python/).

In [7]:
for k, v in response.headers.items():
    print(f'{k}: {v}')

Content-Type: application/json;charset=utf-8
Transfer-Encoding: chunked
Connection: keep-alive
Date: Thu, 10 Jul 2025 15:58:15 GMT
X-Frame-Options: SAMEORIGIN
Access-Control-Allow-Origin: *
X-XSS-Protection: 1; mode=block
CMR-Request-Id: 8ff052e8-112b-41b2-85b2-58d761bd90d2
Content-SHA1: 7329dbb083192cce88b80492dd18b240de504e02
Strict-Transport-Security: max-age=31536000; includeSubDomains; preload
CMR-Search-After: [0.0,8800.0,"ATL03","6",2596864127,68]
CMR-Hits: 7479
Access-Control-Expose-Headers: CMR-Hits, CMR-Request-Id, X-Request-Id, CMR-Scroll-Id, CMR-Search-After, CMR-Timed-Out, CMR-Shapefile-Original-Point-Count, CMR-Shapefile-Simplified-Point-Count
X-Content-Type-Options: nosniff
CMR-Took: 570
X-Request-Id: 7y2lVaP6uEuz-38pqCJ9u1yfxXNycSQv8aMWsPe8DafYu-q0mHKrvA==
Content-MD5: 8de7640834a2ba05b24cd861ddd1a2f0
Vary: Accept-Encoding
Content-Encoding: gzip
Server: ServerTokens ProductOnly
X-Cache: Miss from cloudfront
Via: 1.1 0bf3f0b7038d55ea4f178432aa6ddc52.cloudfront.net (Cloud

Each item in the dictionary can be accessed in the normal way you access a `python` dictionary but the keys uniquely case-**in**sensitive. Let's take a look at the commonly used `CMR-Hits` key.



In [8]:
response.headers['CMR-Hits']

'7479'

Note that "cmr-hits" works as well!

In [9]:
response.headers['cmr-hits']

'7479'

## Query CMR Using Provider 

In some situations the response to your query can return a very large number of result, some of which may not be relevant. We can add additional [query parameters](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html) to restrict the information returned. We're going to restrict the search by the `provider` parameter.

You can modify the code below to explore all Earthdata data products hosted by the various providers. When searching by provider, use _Cloud Provider_ to search for cloud-hosted datasets and _On-Premises Provider_ to search for datasets archived at the DAACs. A partial list of providers is given below.

DAAC      | Short Name                              | Cloud Provider | On-Premises Provider  
----------|-----------------------------------------|----------------|----------------------  
NSIDC     | National Snow and Ice Data Center       | NSIDC_CPRD     | NSIDC_ECS  
GHRC DAAC | Global Hydrometeorology Resource Center | GHRC_DAAC      | GHRC_DAAC  
PO DAAC   | Physical Oceanography Distributed Active Archive Center | POCLOUD | PODAAC  
ASF       | Alaska Satellite Facility | ASF | ASF  
ORNL DAAC | Oak Ridge National Laboratory | ORNL_CLOUD | ORNL_DAAC  
LP DAAC   | Land Processes Distributed Active Archive Center | LPCLOUD | LPDAAC_ECS
GES DISC  | NASA Goddard Earth Sciences (GES) Data and Information Services Center (DISC) | GES_DISC | GES_DISC
OB DAAC   | NASA's Ocean Biology Distributed Active Archive Center |   | OB_DAAC
SEDAC     | NASA's Socioeconomic Data and Applications Center |   | SEDAC

We'll assign the provider to a variable as a _string_ and insert the variable into the parameter argument in the request. 

In [10]:
provider = 'LPCLOUD'

In [11]:
response = requests.get(url,
                        params={
                            'cloud_hosted': 'True',
                            'has_granules': 'True',
                            'provider': provider,
                        },
                        headers= {
                            'Accept': 'application/json'
                            }
                       )
response

<Response [200]>

Let's see how many collections are available through `LPCOUD` provider.

In [12]:
response.headers['cmr-hits']

'786'

Search results are contained in the __content__ part of the Response object. However, `response.content` returns information in bytes.

In [13]:
response.content

b'{"feed":{"updated":"2025-07-10T16:00:15.098Z","id":"https://cmr.earthdata.nasa.gov:443/search/collections.json?cloud_hosted=True&has_granules=True&provider=LPCLOUD","title":"ECHO dataset metadata","entry":[{"processing_level_id":"3","cloud_hosted":true,"boxes":["-90 -180 90 180"],"has_combine":false,"time_start":"2015-11-28T00:00:00.000Z","version_id":"2.0","updated":"2021-08-24T00:00:00.000Z","dataset_id":"HLS Sentinel-2 Multi-spectral Instrument Surface Reflectance Daily Global 30m v2.0","entry_id":"HLSS30_2.0","has_spatial_subsetting":false,"has_transforms":false,"associations":{"variables":["V3204005138-LPCLOUD","V3204005098-LPCLOUD","V3204005093-LPCLOUD","V3204005126-LPCLOUD","V3204005028-LPCLOUD","V3204004957-LPCLOUD","V3204004918-LPCLOUD","V3204005134-LPCLOUD","V3204005120-LPCLOUD","V3204005143-LPCLOUD","V3204005152-LPCLOUD","V3204005155-LPCLOUD","V3204005111-LPCLOUD","V3204005073-LPCLOUD","V3204005052-LPCLOUD","V3204005146-LPCLOUD","V3204005047-LPCLOUD","V3204005010-LPCLOUD"]

A more convenient way to work with this information is to use `json` formatted data. I'm using pretty print `pprint` to print the data in an easy to read way.    

**Note**
- `response.json()` will format our response in `json` 
- `['feed']['entry']` returns all entries that CMR returned in the request (not the same as __CMR-Hits__)
- `[0]` returns the first entry. Reminder that python starts indexing at 0, not 1!

In [14]:
pprint(response.json()['feed']['entry'][0])

{'archive_center': 'LP DAAC',
 'association_details': {'tools': [{'concept_id': 'TL1860232272-LPDAAC_ECS'}],
                         'variables': [{'concept_id': 'V3204005138-LPCLOUD'},
                                       {'concept_id': 'V3204005098-LPCLOUD'},
                                       {'concept_id': 'V3204005093-LPCLOUD'},
                                       {'concept_id': 'V3204005126-LPCLOUD'},
                                       {'concept_id': 'V3204005028-LPCLOUD'},
                                       {'concept_id': 'V3204004957-LPCLOUD'},
                                       {'concept_id': 'V3204004918-LPCLOUD'},
                                       {'concept_id': 'V3204005134-LPCLOUD'},
                                       {'concept_id': 'V3204005120-LPCLOUD'},
                                       {'concept_id': 'V3204005143-LPCLOUD'},
                                       {'concept_id': 'V3204005152-LPCLOUD'},
                                 

The first response contains a lot more information than we need. We'll narrow in on a few fields to get a feel for what we have. We'll print the name of the dataset (`dataset_id`) and the concept id (`id`). We can build this variable and print statement like we did above with the `url` variable. 

In [15]:
collections = response.json()['feed']['entry']

In [16]:
for collection in collections:
    print(f'{collection["archive_center"]} | {collection["dataset_id"]} | {collection["short_name"]} |{collection["id"]}')

LP DAAC | HLS Sentinel-2 Multi-spectral Instrument Surface Reflectance Daily Global 30m v2.0 | HLSS30 |C2021957295-LPCLOUD
LP DAAC | NASA Shuttle Radar Topography Mission Global 1 arc second V003 | SRTMGL1 |C2763266360-LPCLOUD
LP DAAC | HLS Landsat Operational Land Imager Surface Reflectance and TOA Brightness Daily Global 30m v2.0 | HLSL30 |C2021957657-LPCLOUD
LP DAAC | ASTER Global Digital Elevation Model V003 | ASTGTM |C1711961296-LPCLOUD
LP DAAC | MODIS/Terra Thermal Anomalies/Fire 5-Min L2 Swath 1km V061 | MOD14 |C2271754179-LPCLOUD
LP DAAC | MODIS/Terra Vegetation Indices 16-Day L3 Global 250m SIN Grid V061 | MOD13Q1 |C1748066515-LPCLOUD
LP DAAC | GEDI L2A Elevation and Height Metrics Data Global Footprint Level V002 | GEDI02_A |C2142771958-LPCLOUD
LP DAAC | MODIS/Aqua Thermal Anomalies/Fire 5-Min L2 Swath 1km V061 | MYD14 |C2278858993-LPCLOUD
LP DAAC | EMIT L2A Estimated Surface Reflectance and Uncertainty and Masks 60 m V001 | EMITL2ARFL |C2408750690-LPCLOUD
LP DAAC | ECOSTRESS

`CMR-Hits` showed 209 data collections above but CMR restricts the number of results returned by each query. The default is 10 but it can be set to a maximum of 2000 by adding `page_size` parameter. We can set the `page_size` parameter to 300 (higher than the number of results returned) so we get all results in a single query.

In [17]:
response = requests.get(url,
                        params={
                            'cloud_hosted': 'True',
                            'has_granules': 'True',
                            'provider': provider,
                            'page_size': 300
                        },
                        headers={
                            'Accept': 'application/json'
                            }
                       )
response

<Response [200]>

Now, when we can re-run our for loop for the collections we now have all of the available collections listed.

In [18]:
collections = response.json()['feed']['entry']
for collection in collections:
    print(f'{collection["archive_center"]} | {collection["dataset_id"]} | {collection["short_name"]} |{collection["id"]}')

LP DAAC | HLS Sentinel-2 Multi-spectral Instrument Surface Reflectance Daily Global 30m v2.0 | HLSS30 |C2021957295-LPCLOUD
LP DAAC | NASA Shuttle Radar Topography Mission Global 1 arc second V003 | SRTMGL1 |C2763266360-LPCLOUD
LP DAAC | HLS Landsat Operational Land Imager Surface Reflectance and TOA Brightness Daily Global 30m v2.0 | HLSL30 |C2021957657-LPCLOUD
LP DAAC | ASTER Global Digital Elevation Model V003 | ASTGTM |C1711961296-LPCLOUD
LP DAAC | MODIS/Terra Thermal Anomalies/Fire 5-Min L2 Swath 1km V061 | MOD14 |C2271754179-LPCLOUD
LP DAAC | MODIS/Terra Vegetation Indices 16-Day L3 Global 250m SIN Grid V061 | MOD13Q1 |C1748066515-LPCLOUD
LP DAAC | GEDI L2A Elevation and Height Metrics Data Global Footprint Level V002 | GEDI02_A |C2142771958-LPCLOUD
LP DAAC | MODIS/Aqua Thermal Anomalies/Fire 5-Min L2 Swath 1km V061 | MYD14 |C2278858993-LPCLOUD
LP DAAC | EMIT L2A Estimated Surface Reflectance and Uncertainty and Masks 60 m V001 | EMITL2ARFL |C2408750690-LPCLOUD
LP DAAC | ECOSTRESS

# Searching for Granules
In NASA speak, Granules are files or groups of files. We will search by `concept_id`, `temporal`, and `bounding_box`.  Details about these search parameters can be found in the CMR API Documentation.

We need to change the resource url to look for __granules__ instead of collections

In [19]:
url = f'{CMR_OPS}/{"granules"}'
url

'https://cmr.earthdata.nasa.gov/search/granules'

Below, `concept_id`, `temporal`, and `bounding_box` parameters are stored as a string to variables. Collection IDs are what we found from collection query. Here, we used `C2076090826-LPCLOUD` which is ID for ECOSTRESS Tiled Land Surface Temperature and Emissivity Instantaneous L2 Global 70 m V002 ([ECO_L2T_LSTE](https://doi.org/10.5067/ECOSTRESS/ECO_L2T_LSTE.002)) as an example.

The formatting of the values for each parameter is quite specific. For `temporal` and `bounding_box` follow the format below:
__Temporal parameters__ are in ISO 8061 format `yyyy-MM-ddTHH:mm:ssZ`.  
__Bounding box coordinates__ are lower left longitude, lower left latitude, upper right longitude, upper right latitude. 

In [24]:
collection_id = 'C1748058432-LPCLOUD' # MOD11A1
date_range = '2000-01-01T00:00:00Z,2025-07-11T23:59:59Z'
bbox = '2.69170169436,4.24059418377,14.5771777686,13.8659239771'


In [25]:
response = requests.get(url, 
                        params={
                            'concept_id': collection_id,
                            'temporal': date_range,
                            'bounding_box': bbox,
                            'page_size': 200
                            },
                        headers={
                            'Accept': 'application/json'
                            }
                       )
print(response.status_code)

200


Let's see how many granules are found for this query. 

In [26]:
print(response.headers['CMR-Hits'])

36758


Now, look at the the first granule metadata. 

In [34]:
granules = response.json()['feed']['entry']
pprint(granules[0])

{'browse_flag': True,
 'cloud_cover': '64.0',
 'collection_concept_id': 'C1748058432-LPCLOUD',
 'coordinate_system': 'GEODETIC',
 'data_center': 'LPCLOUD',
 'dataset_id': 'MODIS/Terra Land Surface Temperature/Emissivity Daily L3 '
               'Global 1km SIN Grid V061',
 'day_night_flag': 'BOTH',
 'granule_size': '3.03519',
 'id': 'G2182561954-LPCLOUD',
 'links': [{'href': 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MOD11A1.061/MOD11A1.A2000055.h18v07.061.2020043120835/MOD11A1.A2000055.h18v07.061.2020043120835.hdf',
            'hreflang': 'en-US',
            'rel': 'http://esipfed.org/ns/fedsearch/1.1/data#',
            'title': 'Download MOD11A1.A2000055.h18v07.061.2020043120835.hdf'},
           {'href': 's3://lp-prod-protected/MOD11A1.061/MOD11A1.A2000055.h18v07.061.2020043120835/MOD11A1.A2000055.h18v07.061.2020043120835.hdf',
            'hreflang': 'en-US',
            'rel': 'http://esipfed.org/ns/fedsearch/1.1/s3#',
            'title': 'This link provid

Below, the information for granules are printed. Looking at the granules' spatial box and start time shows they are filtered spatially and temporally for this collection.

In [35]:
for granule in granules:
    print(f'{granule["data_center"]} | {granule["title"]} | {granule["id"]} | {granule["time_start"]} | {granule["polygons"]}')

LPCLOUD | MOD11A1.A2000055.h18v07.061.2020043120835 | G2182561954-LPCLOUD | 2000-02-24T00:00:00.000Z | [['10.0041667 -0.0042208 10.0041667 10.1502892 19.9958333 10.6373196 19.9958333 -0.0044399 10.0041667 -0.0042208']]
LPCLOUD | MOD11A1.A2000055.h19v07.061.2020043120933 | G2182587385-LPCLOUD | 2000-02-24T00:00:00.000Z | [['10.0041667 10.1503061 10.0041667 20.3048161 19.9958333 21.2790969 19.9958333 10.6373374 10.0041667 10.1503061']]
LPCLOUD | MOD11A1.A2000055.h18v08.061.2020043121044 | G2182599082-LPCLOUD | 2000-02-24T00:00:00.000Z | [['0.0041667 -0.0041592 0.0041667 9.9958244 9.9958333 10.1500166 9.9958333 -0.0042323 0.0041667 -0.0041592']]
LPCLOUD | MOD11A1.A2000055.h19v08.061.2020043120847 | G2182650440-LPCLOUD | 2000-02-24T00:00:00.000Z | [['0.0041667 9.995841 0.0041667 19.9958246 9.9958333 20.3042824 9.9958333 10.1500335 0.0041667 9.995841']]
LPCLOUD | MOD11A1.A2000056.h18v07.061.2020043120835 | G2182562134-LPCLOUD | 2000-02-25T00:00:00.000Z | [['10.0041667 -0.0042208 10.0041667 

## Get URLs to cloud data assets

Now that we have a list of granules filtered spatially and temporally for our collection, we can save the links to access the data. 
Below, HTTPS and S3 links are stored in two different lists. HTTPS links can be used to access data locally while S3 links can be used to access data in the cloud. View [LP DAAC Data Resources](hub.com/nasa/LPDAAC-Data-Resources) for resources available for accessing and working with data collections in the Earthdata Cloud.

In [47]:
https_urls = []
for granule in granules:
    https_urls.extend([l['href'] for l in granule['links'] if 'https' in l['href'] and '.hdf' in l['href']])

https_urls

['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MOD11A1.061/MOD11A1.A2000055.h18v07.061.2020043120835/MOD11A1.A2000055.h18v07.061.2020043120835.hdf',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MOD11A1.061/MOD11A1.A2000055.h19v07.061.2020043120933/MOD11A1.A2000055.h19v07.061.2020043120933.hdf',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MOD11A1.061/MOD11A1.A2000055.h18v08.061.2020043121044/MOD11A1.A2000055.h18v08.061.2020043121044.hdf',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MOD11A1.061/MOD11A1.A2000055.h19v08.061.2020043120847/MOD11A1.A2000055.h19v08.061.2020043120847.hdf',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MOD11A1.061/MOD11A1.A2000056.h18v07.061.2020043120835/MOD11A1.A2000056.h18v07.061.2020043120835.hdf',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MOD11A1.061/MOD11A1.A2000056.h19v07.061.2020043120933/MOD11A1.A2000056.h19v07.061.2020043120933.hdf',
 'ht

In [48]:
s3_urls = []
for granule in granules:
    s3_urls.extend([l['href'] for l in granule['links'] if 's3' in l['href'] and '.hdf' in l['href']])

s3_urls

['s3://lp-prod-protected/MOD11A1.061/MOD11A1.A2000055.h18v07.061.2020043120835/MOD11A1.A2000055.h18v07.061.2020043120835.hdf',
 's3://lp-prod-protected/MOD11A1.061/MOD11A1.A2000055.h19v07.061.2020043120933/MOD11A1.A2000055.h19v07.061.2020043120933.hdf',
 's3://lp-prod-protected/MOD11A1.061/MOD11A1.A2000055.h18v08.061.2020043121044/MOD11A1.A2000055.h18v08.061.2020043121044.hdf',
 's3://lp-prod-protected/MOD11A1.061/MOD11A1.A2000055.h19v08.061.2020043120847/MOD11A1.A2000055.h19v08.061.2020043120847.hdf',
 's3://lp-prod-protected/MOD11A1.061/MOD11A1.A2000056.h18v07.061.2020043120835/MOD11A1.A2000056.h18v07.061.2020043120835.hdf',
 's3://lp-prod-protected/MOD11A1.061/MOD11A1.A2000056.h19v07.061.2020043120933/MOD11A1.A2000056.h19v07.061.2020043120933.hdf',
 's3://lp-prod-protected/MOD11A1.061/MOD11A1.A2000056.h18v08.061.2020043121044/MOD11A1.A2000056.h18v08.061.2020043121044.hdf',
 's3://lp-prod-protected/MOD11A1.061/MOD11A1.A2000056.h19v08.061.2020043120847/MOD11A1.A2000056.h19v08.061.2020